# Task II: NVE molecular dynamics simulations (Part III, Week05)

Set up the kernel

<center><img src="figures/fig1.png" width=1100 height=240 /></center>

<center><img src="figures/fig2.png" width=350 height=240 /></center>

run the following cells using `shift` + `enter`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import typing as ty
from scipy.stats import linregress
from utilities import plot_data2
from MD import *

# Goals
The obejective of this exercise is estimating the diffusion coefﬁcient, D, using two different
methods, namely: 
- Step 3.1 Einstein’s relation
- Step 3.2 The time integral of the velocity autocorrelation function
- Step 3.3 (Optional) Repeat the same steps for a system at a different temperature and/or density

## Compute MSD and VACF

Now we move to the study of a dynamical quantity which is easily accessible through MD
simulations: the diffusion coefﬁcient. As you have learned during the class, this quantity
can be computed from the mean square displacement (MSD) of the atomic positions through
Einstein’s relation, or from the integral of the velocity autocorrelation function (VACF)

For Einstein’s relation, it
$$MSD(t) = C + 6Dt$$
involves the sampling of the mean square displacement (MSD) and a ﬁtting procedure giving C and D

## Step 3.1 and 3.2 combined 

In [ ]:
nsteps = 450
dt = 0.002
Nruns = 12
cut = 200

# Step up an array for time axis
t = np.linspace(1,nsteps,nsteps)*dt

# Empty variable for output
msd_mean=np.zeros(nsteps)
vacf_mean=np.zeros(nsteps)
D_msd=[]
D_vacf=[]

# Read equilibrated structure from a file
N, L, pos, vel = read_pos_vel('sampleT94.4.dat')
output = {'pos':pos, 'vel': vel}

for i in range(Nruns):

    # Run MD simulation
    output = run_NVE(output['pos'], output['vel'], L, nsteps, N, dt)
    msd_mean += output['msd']/Nruns
    vacf_mean += output['vacf']/Nruns

    # MSD
    slope, intercept, r, p, se = linregress(t[cut:],output['msd'][cut:])
    D_msd.append(slope/6*3.4E-10**2*4.6286E+11*1E+4)

    # VACF
    D_vacf.append(sum(output['vacf']*dt*3.4**2*4.6286*10**-5))

    # Write MSD and VACF into a file
    np.savetxt('sample.dat%i.samp'%i, np.column_stack((t, output['msd'] ,output['vacf'])))


# Plot averaged MSD
plt.plot(t,msd_mean)
slope, intercept, r, p, se = linregress(t[cut:],msd_mean[cut:])
plt.plot(t,slope*t+intercept, color='red')
plt.show()

# Plot averaged VACF
plt.plot(t,vacf_mean)
plt.show()

# Write diffusion coefficients into a file
np.savetxt('Dcoeff.dat', np.column_stack((D_msd, D_vacf)))


## Step 3.3 (Optional)

<div class="alert alert-block alert-info"><b>TODO:</b> Repeat the same steps for a system at a different temperature and/or density
(remember that the accuracy of the time integration may depend on these parameters) and
compare the static (step2.ipynb) and dynamic properties. </div>

In [ ]:
# your simulations here
# you can copy the codes in step1.ipynb

## Important remarks
1. ensemble average needs either to average on different time origins in the same run (not
implemented) or to average (at same times) the quantity obtained as a function of time in
several different runs (of same length). Smaller systems (small N) are subject to larger
statistical ﬂuctuations, therefore the deviation from the ensemble average may be quite
large if a single realization is used to estimate the diffusion coefﬁcient.
2. since we are sampling a dynamical quantity, the accuracy in the description of the particle
trajectories plays more important role here. Be careful on the choice of dt; the value used
so far may not be sufﬁciently small.

Step3.ipynb adopts the second method for computing the ensemble average of the dynamical
quantities, i.e. the MSD and VACF are calculated as an average over a number of simulations
deﬁned by Nruns.